In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import zipfile
path = '/content/drive/MyDrive/New.zip'
zip_obj = zipfile.ZipFile(file = path, mode = 'r')
zip_obj.extractall('./')
zip_obj.close()

In [ ]:
import os
Root = "/content/New"
os.chdir(Root)

In [ ]:
ls

speech-emotion-recognition-ravdess-data/


In [ ]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [ ]:
#Extract features (mfcc, chroma, mel) from a sound file
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(y=X, sr=sample_rate))
            result=np.hstack((result, mel))
    return result

In [ ]:
# Emotions in the RAVDESS dataset
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

#Emotions to observe
observed_emotions=['calm', 'happy', 'fearful', 'disgust']

In [ ]:
#Load the data and extract features for each sound file
def load_data(test_size=0.2):
    x,y=[],[]
    for file in glob.glob("/content/New/speech-emotion-recognition-ravdess-data/Actor_*/*.wav"):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=4)

In [ ]:
#Split the dataset
x_train,x_test,y_train,y_test=load_data(test_size=0.20)

In [ ]:
x_train

array([[-4.66707550e+02,  1.52751408e+01, -2.77987614e+01, ...,
         5.97086847e-01,  4.92325515e-01,  2.70676702e-01],
       [-6.93024231e+02,  5.46327591e+01,  1.00681849e+01, ...,
         7.39980876e-01,  7.03393340e-01,  1.25649222e-03],
       [-5.90802185e+02,  5.89666595e+01, -1.06967535e+01, ...,
         7.18967199e-01,  7.03100920e-01,  1.31369755e-02],
       ...,
       [-5.28567993e+02,  4.68415756e+01, -3.67727685e+00, ...,
         6.57660782e-01,  5.81645250e-01,  7.66419545e-02],
       [-6.39770752e+02,  6.69016953e+01, -9.46646035e-01, ...,
         7.87018120e-01,  7.60452628e-01,  2.37084506e-03],
       [-5.39380310e+02,  3.26435585e+01, -1.24525795e+01, ...,
         5.90659440e-01,  5.83834946e-01,  1.62012443e-01]])

In [ ]:
print((x_train.shape[0], x_test.shape[0]))

(614, 154)


In [ ]:
print(f'Features extracted: {x_train.shape[1]}')

Features extracted: 53


In [ ]:
#Initialize the Multi Layer Perceptron Classifier
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)

In [ ]:
model.fit(x_train,y_train)

MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=500)

In [ ]:
#Predict for the test set
y_pred=model.predict(x_test)

In [ ]:
y_pred

array(['happy', 'happy', 'disgust', 'calm', 'happy', 'disgust', 'happy',
       'happy', 'disgust', 'fearful', 'disgust', 'fearful', 'calm',
       'disgust', 'calm', 'disgust', 'disgust', 'fearful', 'calm',
       'fearful', 'happy', 'calm', 'disgust', 'fearful', 'disgust',
       'disgust', 'calm', 'fearful', 'fearful', 'happy', 'fearful',
       'happy', 'calm', 'fearful', 'happy', 'fearful', 'happy', 'calm',
       'fearful', 'calm', 'calm', 'calm', 'happy', 'calm', 'happy',
       'happy', 'disgust', 'calm', 'happy', 'calm', 'fearful', 'happy',
       'disgust', 'happy', 'fearful', 'fearful', 'happy', 'disgust',
       'calm', 'happy', 'disgust', 'disgust', 'happy', 'calm', 'disgust',
       'calm', 'calm', 'calm', 'calm', 'fearful', 'calm', 'happy', 'calm',
       'happy', 'calm', 'calm', 'calm', 'fearful', 'calm', 'fearful',
       'happy', 'disgust', 'calm', 'fearful', 'disgust', 'calm', 'calm',
       'calm', 'disgust', 'fearful', 'happy', 'fearful', 'fearful',
       'calm', 

In [ ]:
#Calculate the accuracy of our model
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)

#Print the accuracy
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 71.43%


In [ ]:
from sklearn.metrics import accuracy_score, f1_score

In [ ]:
f1_score(y_test, y_pred,average=None)

array([0.79012346, 0.58064516, 0.7804878 , 0.6746988 ])

In [ ]:
import pandas as pd
df=pd.DataFrame({'Actual': y_test, 'Predicted':y_pred})
df.head(25)

,Actual,Predicted
0,happy,happy
1,happy,happy
2,happy,disgust
3,calm,calm
4,happy,happy
5,disgust,disgust
6,happy,happy
7,happy,happy
8,disgust,disgust
9,happy,fearful


In [ ]:
import pickle
# Writing different model files to file
with open( 'modelForPrediction1.sav', 'wb') as f:
    pickle.dump(model,f)

In [ ]:
filename = 'modelForPrediction1.sav'
loaded_model = pickle.load(open(filename, 'rb')) # loading the model file from the storage

feature=extract_feature("/content/New/speech-emotion-recognition-ravdess-data/Actor_01/03-01-01-01-01-01-01.wav", mfcc=True, chroma=True, mel=True)

feature=feature.reshape(1,-1)

prediction=loaded_model.predict(feature)
prediction

array(['fearful'], dtype='<U7')

In [ ]:
feature

array([[-6.93497009e+02,  5.00643921e+01,  5.71450531e-01,
         1.43299656e+01,  3.33637071e+00, -2.54071975e+00,
        -4.05790949e+00, -1.07119999e+01, -7.29413939e+00,
         1.74018872e+00, -4.19064236e+00,  1.95466173e+00,
        -5.24789381e+00,  2.78142977e+00, -3.16756773e+00,
        -3.40008307e+00, -2.37803197e+00, -5.68717539e-01,
        -6.47753334e+00, -1.24320555e+00, -2.80542517e+00,
        -5.43635798e+00, -4.46875393e-01, -3.63516593e+00,
        -2.98372602e+00, -5.63902557e-01, -1.65101993e+00,
        -5.55944741e-01, -3.41018438e+00, -2.24465466e+00,
        -3.13058877e+00, -2.70089960e+00, -1.88821292e+00,
        -5.54154456e-01, -3.96459913e+00, -2.13485193e+00,
        -3.94577074e+00, -1.62457860e+00, -2.03990698e+00,
        -3.62910867e+00,  6.33734703e-01,  6.48760676e-01,
         6.21744275e-01,  6.34553194e-01,  6.60345614e-01,
         6.60463989e-01,  7.00949013e-01,  7.31556356e-01,
         7.46921539e-01,  7.23420441e-01,  7.16119647e-0